# Keras CNN

(Dataset is secret)

### Read Data

In [1]:
from config import *
import cv2
import glob
import pandas as pd

abst_filename_list = [img for img in glob.glob(IMG_DIR + "/*.jpg")]

filename_list = [name.split(IMG_DIR + "\\")[1] for name in abst_filename_list]
filename_list = [name.split(".jpg")[0] for name in filename_list]

model_id_list = list()
product_id_list = list()
image_id_list = list()

for info in [filename.split("_") for filename in filename_list]:
    model_id_list.append(info[0])
    product_id_list.append(info[1])
    image_id_list.append(info[2])
    
dataset_list = list(zip(abst_filename_list, filename_list, model_id_list, product_id_list, image_id_list))

df = pd.DataFrame(dataset_list, columns=["absolute_path", "file_name", "model_id", "product_id_list", "image_id_list"])

del abst_filename_list, model_id_list, product_id_list, image_id_list, filename_list, dataset_list

In [2]:
converted_id_dict = dict()
converted_id_list = list()

labels = [int(id) for id in df["model_id"]]
labels_set = list(set(labels))

for i, id_ in enumerate(labels_set):
    converted_id_dict[id_] = i

for label in labels:
    converted_id_list.append(converted_id_dict.get(label))

df["label"] = converted_id_list

In [3]:
df.head()

,absolute_path,file_name,model_id,product_id_list,image_id_list,label
0,../dataset\10047545284_10226223039_0.jpg,10047545284_10226223039_0,10047545284,10226223039,0,134
1,../dataset\10047545284_10226256889_0.jpg,10047545284_10226256889_0,10047545284,10226256889,0,134
2,../dataset\10047545284_10226293526_0.jpg,10047545284_10226293526_0,10047545284,10226293526,0,134
3,../dataset\10047545284_10321296333_0.jpg,10047545284_10321296333_0,10047545284,10321296333,0,134
4,../dataset\10047545284_10321296355_0.jpg,10047545284_10321296355_0,10047545284,10321296355,0,134


### Read Features

In [4]:
import os
import numpy as np

import tensorflow.keras as k

In [5]:
dataset = list()
label_dataset = list()

img_size = 50

for path in df["absolute_path"]:
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_CUBIC)
    dataset.append(np.array(img))
    
reshaped_dataset = np.array([np.reshape(data, (img_size, img_size, 3)) for data in dataset])
label_dataset = np.array([[label] for label in df["label"]])

In [6]:
print("Dataset Shape : {}".format(reshaped_dataset.shape))
print("Label Shape : {}".format(label_dataset.shape))

Dataset Shape : (10618, 50, 50, 3)
Label Shape : (10618, 1)


In [7]:
num_classes = len(labels_set)

label_dataset_one_hot = k.utils.to_categorical(label_dataset, num_classes)

In [8]:
reshaped_dataset = reshaped_dataset.astype("float32")
reshaped_dataset /= 255.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    reshaped_dataset,
    label_dataset_one_hot,
    test_size=0.3,
    random_state=1
)

### Making Model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [11]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=X_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
__________

In [13]:
epochs = 670
batch_size = 128
lr = 0.0001
decay = 1e-6

optimizer = k.optimizers.RMSprop(lr=lr, decay=decay)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
history = model.fit(
    X_train, 
    y_train, 
    batch_size=batch_size, 
    epochs=epochs, validation_data=(X_test, y_test), 
    shuffle=True, 
    verbose=2
)

Train on 7432 samples, validate on 3186 samples
Epoch 1/670
 - 11s - loss: 5.0135 - acc: 0.0188 - val_loss: 4.9767 - val_acc: 0.0945
Epoch 2/670
 - 3s - loss: 4.8771 - acc: 0.0583 - val_loss: 4.6458 - val_acc: 0.1321
Epoch 3/670
 - 3s - loss: 4.4526 - acc: 0.1308 - val_loss: 4.0004 - val_acc: 0.1861
Epoch 4/670
 - 3s - loss: 3.9500 - acc: 0.2114 - val_loss: 3.3419 - val_acc: 0.3415
Epoch 5/670
 - 3s - loss: 3.4878 - acc: 0.2815 - val_loss: 2.9226 - val_acc: 0.5044
Epoch 6/670
 - 3s - loss: 3.1037 - acc: 0.3463 - val_loss: 2.4408 - val_acc: 0.5832
Epoch 7/670
 - 3s - loss: 2.7305 - acc: 0.4170 - val_loss: 1.8874 - val_acc: 0.6582
Epoch 8/670
 - 3s - loss: 2.4045 - acc: 0.4742 - val_loss: 1.6289 - val_acc: 0.6921
Epoch 9/670
 - 3s - loss: 2.1250 - acc: 0.5389 - val_loss: 1.3615 - val_acc: 0.7483
Epoch 10/670
 - 3s - loss: 1.8982 - acc: 0.5770 - val_loss: 1.1665 - val_acc: 0.7674
Epoch 11/670
 - 3s - loss: 1.7039 - acc: 0.6128 - val_loss: 1.0536 - val_acc: 0.7963
Epoch 12/670
 - 3s - loss

Epoch 97/670
 - 3s - loss: 0.0475 - acc: 0.9851 - val_loss: 0.4613 - val_acc: 0.9288
Epoch 98/670
 - 3s - loss: 0.0461 - acc: 0.9879 - val_loss: 0.4470 - val_acc: 0.9294
Epoch 99/670
 - 3s - loss: 0.0397 - acc: 0.9888 - val_loss: 0.4561 - val_acc: 0.9281
Epoch 100/670
 - 3s - loss: 0.0394 - acc: 0.9878 - val_loss: 0.4501 - val_acc: 0.9325
Epoch 101/670
 - 3s - loss: 0.0407 - acc: 0.9894 - val_loss: 0.4666 - val_acc: 0.9338
Epoch 102/670
 - 3s - loss: 0.0387 - acc: 0.9887 - val_loss: 0.4597 - val_acc: 0.9303
Epoch 103/670
 - 3s - loss: 0.0387 - acc: 0.9890 - val_loss: 0.4660 - val_acc: 0.9269
Epoch 104/670
 - 3s - loss: 0.0324 - acc: 0.9890 - val_loss: 0.4951 - val_acc: 0.9319
Epoch 105/670
 - 3s - loss: 0.0404 - acc: 0.9896 - val_loss: 0.4796 - val_acc: 0.9313
Epoch 106/670
 - 3s - loss: 0.0415 - acc: 0.9886 - val_loss: 0.4771 - val_acc: 0.9306
Epoch 107/670
 - 3s - loss: 0.0345 - acc: 0.9888 - val_loss: 0.4740 - val_acc: 0.9328
Epoch 108/670
 - 3s - loss: 0.0326 - acc: 0.9902 - val_lo

Epoch 193/670
 - 3s - loss: 0.0262 - acc: 0.9918 - val_loss: 0.4735 - val_acc: 0.9278
Epoch 194/670
 - 3s - loss: 0.0252 - acc: 0.9929 - val_loss: 0.4704 - val_acc: 0.9288
Epoch 195/670
 - 3s - loss: 0.0246 - acc: 0.9919 - val_loss: 0.4967 - val_acc: 0.9300
Epoch 196/670
 - 3s - loss: 0.0271 - acc: 0.9919 - val_loss: 0.4841 - val_acc: 0.9335
Epoch 197/670
 - 3s - loss: 0.0254 - acc: 0.9913 - val_loss: 0.5011 - val_acc: 0.9294
Epoch 198/670
 - 3s - loss: 0.0273 - acc: 0.9919 - val_loss: 0.5047 - val_acc: 0.9291
Epoch 199/670
 - 3s - loss: 0.0236 - acc: 0.9921 - val_loss: 0.5153 - val_acc: 0.9288
Epoch 200/670
 - 3s - loss: 0.0241 - acc: 0.9933 - val_loss: 0.4852 - val_acc: 0.9266
Epoch 201/670
 - 3s - loss: 0.0256 - acc: 0.9921 - val_loss: 0.4959 - val_acc: 0.9300
Epoch 202/670
 - 3s - loss: 0.0269 - acc: 0.9923 - val_loss: 0.4941 - val_acc: 0.9275
Epoch 203/670
 - 3s - loss: 0.0258 - acc: 0.9919 - val_loss: 0.5133 - val_acc: 0.9281
Epoch 204/670
 - 3s - loss: 0.0259 - acc: 0.9923 - val

Epoch 289/670
 - 3s - loss: 0.0249 - acc: 0.9926 - val_loss: 0.4587 - val_acc: 0.9300
Epoch 290/670
 - 3s - loss: 0.0242 - acc: 0.9931 - val_loss: 0.5256 - val_acc: 0.9306
Epoch 291/670
 - 3s - loss: 0.0235 - acc: 0.9943 - val_loss: 0.5845 - val_acc: 0.9291
Epoch 292/670
 - 3s - loss: 0.0264 - acc: 0.9915 - val_loss: 0.4977 - val_acc: 0.9275
Epoch 293/670
 - 3s - loss: 0.0245 - acc: 0.9929 - val_loss: 0.4915 - val_acc: 0.9281
Epoch 294/670
 - 3s - loss: 0.0264 - acc: 0.9921 - val_loss: 0.5288 - val_acc: 0.9291
Epoch 295/670
 - 3s - loss: 0.0225 - acc: 0.9931 - val_loss: 0.4987 - val_acc: 0.9288
Epoch 296/670
 - 3s - loss: 0.0215 - acc: 0.9937 - val_loss: 0.4877 - val_acc: 0.9284
Epoch 297/670
 - 3s - loss: 0.0215 - acc: 0.9930 - val_loss: 0.4875 - val_acc: 0.9291
Epoch 298/670
 - 3s - loss: 0.0293 - acc: 0.9909 - val_loss: 0.5282 - val_acc: 0.9281
Epoch 299/670
 - 3s - loss: 0.0223 - acc: 0.9937 - val_loss: 0.5042 - val_acc: 0.9306
Epoch 300/670
 - 3s - loss: 0.0228 - acc: 0.9927 - val

Epoch 385/670
 - 3s - loss: 0.0242 - acc: 0.9926 - val_loss: 0.3910 - val_acc: 0.9278
Epoch 386/670
 - 3s - loss: 0.0255 - acc: 0.9926 - val_loss: 0.4878 - val_acc: 0.9303
Epoch 387/670
 - 3s - loss: 0.0259 - acc: 0.9923 - val_loss: 0.4969 - val_acc: 0.9281
Epoch 388/670
 - 3s - loss: 0.0220 - acc: 0.9934 - val_loss: 0.5374 - val_acc: 0.9331
Epoch 389/670
 - 3s - loss: 0.0232 - acc: 0.9927 - val_loss: 0.4499 - val_acc: 0.9284
Epoch 390/670
 - 3s - loss: 0.0219 - acc: 0.9935 - val_loss: 0.4144 - val_acc: 0.9275
Epoch 391/670
 - 3s - loss: 0.0289 - acc: 0.9917 - val_loss: 0.5610 - val_acc: 0.9316
Epoch 392/670
 - 3s - loss: 0.0249 - acc: 0.9921 - val_loss: 0.5487 - val_acc: 0.9331
Epoch 393/670
 - 3s - loss: 0.0273 - acc: 0.9909 - val_loss: 0.4050 - val_acc: 0.9259
Epoch 394/670
 - 3s - loss: 0.0254 - acc: 0.9921 - val_loss: 0.5152 - val_acc: 0.9322
Epoch 395/670
 - 3s - loss: 0.0241 - acc: 0.9930 - val_loss: 0.4036 - val_acc: 0.9294
Epoch 396/670
 - 3s - loss: 0.0242 - acc: 0.9917 - val

Epoch 481/670
 - 3s - loss: 0.0226 - acc: 0.9937 - val_loss: 0.4517 - val_acc: 0.9341
Epoch 482/670
 - 3s - loss: 0.0231 - acc: 0.9938 - val_loss: 0.4512 - val_acc: 0.9335
Epoch 483/670
 - 3s - loss: 0.0233 - acc: 0.9927 - val_loss: 0.3850 - val_acc: 0.9294
Epoch 484/670
 - 3s - loss: 0.0224 - acc: 0.9929 - val_loss: 0.5241 - val_acc: 0.9300
Epoch 485/670
 - 3s - loss: 0.0225 - acc: 0.9934 - val_loss: 0.5055 - val_acc: 0.9341
Epoch 486/670
 - 3s - loss: 0.0260 - acc: 0.9917 - val_loss: 0.6302 - val_acc: 0.9335
Epoch 487/670
 - 3s - loss: 0.0269 - acc: 0.9922 - val_loss: 0.4231 - val_acc: 0.9300
Epoch 488/670
 - 3s - loss: 0.0191 - acc: 0.9938 - val_loss: 0.5676 - val_acc: 0.9344
Epoch 489/670
 - 3s - loss: 0.0215 - acc: 0.9926 - val_loss: 0.5156 - val_acc: 0.9319
Epoch 490/670
 - 3s - loss: 0.0209 - acc: 0.9929 - val_loss: 0.5266 - val_acc: 0.9328
Epoch 491/670
 - 3s - loss: 0.0220 - acc: 0.9933 - val_loss: 0.4973 - val_acc: 0.9319
Epoch 492/670
 - 3s - loss: 0.0232 - acc: 0.9929 - val

Epoch 577/670
 - 3s - loss: 0.0255 - acc: 0.9923 - val_loss: 0.5329 - val_acc: 0.9300
Epoch 578/670
 - 3s - loss: 0.0233 - acc: 0.9938 - val_loss: 0.5374 - val_acc: 0.9328
Epoch 579/670
 - 3s - loss: 0.0204 - acc: 0.9941 - val_loss: 0.5199 - val_acc: 0.9353
Epoch 580/670
 - 3s - loss: 0.0231 - acc: 0.9927 - val_loss: 0.4746 - val_acc: 0.9325
Epoch 581/670
 - 3s - loss: 0.0273 - acc: 0.9915 - val_loss: 0.5321 - val_acc: 0.9344
Epoch 582/670
 - 3s - loss: 0.0200 - acc: 0.9925 - val_loss: 0.6167 - val_acc: 0.9353
Epoch 583/670
 - 3s - loss: 0.0270 - acc: 0.9911 - val_loss: 0.4804 - val_acc: 0.9360
Epoch 584/670
 - 3s - loss: 0.0243 - acc: 0.9921 - val_loss: 0.4985 - val_acc: 0.9300
Epoch 585/670
 - 3s - loss: 0.0205 - acc: 0.9926 - val_loss: 0.4251 - val_acc: 0.9297
Epoch 586/670
 - 3s - loss: 0.0242 - acc: 0.9926 - val_loss: 0.5810 - val_acc: 0.9341
Epoch 587/670
 - 3s - loss: 0.0270 - acc: 0.9917 - val_loss: 0.5118 - val_acc: 0.9309
Epoch 588/670
 - 3s - loss: 0.0241 - acc: 0.9930 - val

In [15]:
print("Accuracy : %.2f" % history.history["acc"][-1])
print("Valid Accuracy : %.2f" % history.history["val_acc"][-1])

Accuracy : 0.99
Valid Accuracy : 0.93


In [16]:
# Saving Model and Weights
model_json = model.to_json()
with open(DATA_DIR + "/d2_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights(DATA_DIR + "/d2_weights.h5")

### Predict Data

In [17]:
y_pred = model.predict(reshaped_dataset)
y_pred_label = np.argmax(y_pred, axis=1)

In [18]:
y_pred_label

array([134, 134, 134, ..., 111, 111, 111], dtype=int64)

### Save to files

In [19]:
# Writing to .txt
f = open(DATA_DIR + "/labels_pred.txt", "w")
for label in y_pred_label:
    label_input = str(label)
    f.write(label_input + "\n")
f.close()

In [20]:
# Writing to .csv (with clustering)
csv_dict = {"file_name": df["file_name"], "labels_pred": y_pred_label}
df_csv = pd.DataFrame(data=csv_dict)
df_csv.to_csv(DATA_DIR + "/pred_db.csv")